### Imports

In [1]:
%run utils.ipynb

import os
import time
import math 
import random

import numpy as np
import scipy.sparse as sp
import tensorflow as tf
import numpy.random as rand
import keras.backend as K



from scipy.sparse import linalg as spla
from tensorflow.keras import layers, models, Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Input, Dropout, Concatenate, Embedding, Lambda, Reshape
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, History
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.regularizers import l1_l2
from collections import deque
from sklearn.neighbors import KernelDensity
from scipy.stats import pearsonr

### LINE+

In [2]:
def preprocess_nxgraph(graph):
    node2idx = {}
    idx2node = []
    node_size = 0
    for node in graph.nodes():
        node2idx[node] = node_size
        idx2node.append(node)
        node_size += 1
    return idx2node, node2idx


def main_loss(alpha):
    def line_loss(y_true, y_pred):
        return alpha * -K.mean(K.log(K.sigmoid(y_true * y_pred)))
    return line_loss

def l_ortho_line(gamma):
    
    def loss_3rd(y_true, y_pred):
        
        E = y_pred
        A = y_true
        
        batch_size = tf.cast(K.shape(A)[0], np.float32)
    
        return gamma * E / batch_size

    
    return loss_3rd

def l_sparse_line(delta):
    
    def loss_4th(y_true, y_pred):
        
        E = y_pred
        sense = y_true
                
        batch_size = tf.cast(K.shape(E)[0], np.float32)
            
        return delta * tf.reduce_sum(tf.norm(E, ord = 1, axis = 0)) / batch_size
    
    return loss_4th

def create_model_line(num_nodes, embedding_size, sense_feat_size, order = 'second', batch_size = 128):

    v_i = Input(shape = (1,))
    v_j = Input(shape = (1,))
    adj = Input(shape = (None, ))
    sense_i = Input(shape = (sense_feat_size, ))
    

    first_emb = Embedding(num_nodes, embedding_size, name = 'first_emb')
    second_emb = Embedding(num_nodes, embedding_size, name = 'second_emb')
    context_emb = Embedding(num_nodes, embedding_size, name = 'context_emb')

    v_i_emb = first_emb(v_i)
    v_j_emb = first_emb(v_j)

    v_i_emb_second = second_emb(v_i)
    v_j_context_emb = context_emb(v_j)

    ### First Embed ###
    first = Lambda(lambda x: tf.reduce_sum(x[0] * x[1],
                                               axis = -1,
                                               keepdims = False),
                       name = 'first_order')([v_i_emb, v_j_emb])
    if order == 'first':
    
        first_embed = Reshape((embedding_size,), name = 'ortho_1')(v_i_emb)
        sense_mat = tf.einsum('ij, ik -> ijk', first_embed, sense_i)
        E = sense_mat
        y_norm = tf.linalg.diag_part(tf.matmul(first_embed, first_embed, transpose_b = True), k = 0)
        sense_norm = tf.linalg.diag_part(tf.matmul(sense_i, sense_i, transpose_b = True), k = 0)
        norm = tf.multiply(y_norm, sense_norm)
        E = tf.transpose(tf.transpose(E) / norm)

    
    ### Second Embed ###
    second = Lambda(lambda x: tf.reduce_sum(x[0] * x[1],
                                                axis = -1,
                                                keepdims = False),
                        name = 'second_order')([v_i_emb_second, v_j_context_emb])
    if order == 'second':
        
        second_embed = Reshape((embedding_size,), name = 'ortho_2')(v_i_emb_second)

        sense_mat = tf.einsum('ij, ik -> ijk', second_embed, sense_i)
        E = sense_mat
        y_norm = tf.linalg.diag_part(tf.matmul(second_embed, second_embed, transpose_b = True), k = 0)
        sense_norm = tf.linalg.diag_part(tf.matmul(sense_i, sense_i, transpose_b = True), k = 0)
        norm = tf.multiply(y_norm, sense_norm)
        E = tf.transpose(tf.transpose(E) / norm)
    
    ### Loss Computations
    E_t = tf.transpose(E, perm = [0, 2, 1])

    E_1 = tf.einsum('aij, ajh -> aih', E, E_t)
    E_1 = tf.reduce_sum(E_1)
    
    E_2 = tf.multiply(1.0, E, name = 'sparse_loss')
    
    ####

    if order == 'first':
        output_list = [first_embed, E_1, E_2]
    
    elif order == 'second':
        output_list = [second_embed, E_1, E_2]
    
    else:
        output_list = [first_embed, second_embed, [E_1, E_2], [E_1, E_2]]

    model = Model(inputs = [v_i, v_j, adj, sense_i], outputs = output_list)

    return model, {'first': first_emb, 'second': second_emb}


class LINE:
    def __init__(self, graph, sense_features, alpha, ortho, sparse, learning_rate, batch_size, embedding_size = 8, negative_ratio = 5, order = 'second',):
        """
        :param graph:
        :param embedding_size:
        :param negative_ratio:
        :param order: 'first','second','all'
        """
        if order not in ['first', 'second', 'all']:
            raise ValueError('mode must be first, second or all')

        self.graph = graph
        self.idx2node, self.node2idx = preprocess_nxgraph(graph)
        self.use_alias = True

        self.rep_size = embedding_size
        self.order = order
        self.sense_features = sense_features
        self.sense_feat_size = self.sense_features.shape[1]
        self.alpha = alpha
        self.gamma = ortho
        self.delta = sparse
        self.lr = learning_rate

        self._embeddings = {}
        self.negative_ratio = negative_ratio
        self.order = order
        self.batch_size = batch_size
        
        self.node_size = graph.number_of_nodes()
        self.edge_size = graph.number_of_edges()
        self.samples_per_epoch = self.edge_size * (1 + negative_ratio)

        self._gen_sampling_table()
        self.reset_model()
        
        
        self.A, self.L = self._create_A_L(
            self.graph, self.node2idx)  # Adj Matrix, L Matrix

    def reset_training_config(self, batch_size, times):
        self.batch_size = batch_size
        self.steps_per_epoch = (
            (self.samples_per_epoch - 1) // self.batch_size + 1) * times

    def reset_model(self, opt = 'adam'):

        self.model, self.embedding_dict = create_model_line(self.node_size,
                                                       self.rep_size, 
                                                       self.sense_feat_size,
                                                       self.order, 
                                                       self.batch_size)
        opt = Adam(learning_rate = self.lr, clipnorm = 0.5)
        self.model.compile(opt, [main_loss(self.alpha), l_ortho_line(self.gamma), l_sparse_line(self.delta)])
        self.batch_it = self.batch_iter(self.node2idx)

    def _gen_sampling_table(self):

        # create sampling table for vertex
        power = 0.75
        num_nodes = self.node_size
        node_degree = np.zeros(num_nodes)  # out degree
        node2idx = self.node2idx

        for edge in self.graph.edges():
            node_degree[node2idx[edge[0]]] += self.graph[edge[0]][edge[1]].get('weight', 1.0)

        total_sum = sum([math.pow(node_degree[i], power)
                         for i in range(num_nodes)])
        norm_prob = [float(math.pow(node_degree[j], power)) /
                     total_sum for j in range(num_nodes)]

        self.node_accept, self.node_alias = create_alias_table(norm_prob)

        # create sampling table for edge
        numEdges = self.graph.number_of_edges()
        total_sum = sum([self.graph[edge[0]][edge[1]].get('weight', 1.0)
                         for edge in self.graph.edges()])
        norm_prob = [self.graph[edge[0]][edge[1]].get('weight', 1.0) *
                     numEdges / total_sum for edge in self.graph.edges()]

        self.edge_accept, self.edge_alias = create_alias_table(norm_prob)

    def batch_iter(self, node2idx):

        edges = [(node2idx[x[0]], node2idx[x[1]]) for x in self.graph.edges()]

        data_size = self.graph.number_of_edges()
        shuffle_indices = np.random.permutation(np.arange(data_size))
        # positive or negative mod
        mod = 0
        mod_size = 1 + self.negative_ratio
        h = []
        t = []
        sign = 0
        count = 0
        start_index = 0
        end_index = min(start_index + self.batch_size, data_size)
        while True:
            if mod == 0:

                h = []
                t = []
                for i in range(start_index, end_index):
                    if random.random() >= self.edge_accept[shuffle_indices[i]]:
                        shuffle_indices[i] = self.edge_alias[shuffle_indices[i]]
                    cur_h = edges[shuffle_indices[i]][0]
                    cur_t = edges[shuffle_indices[i]][1]
                    h.append(cur_h)
                    t.append(cur_t)
                sign = np.ones(len(h))
            else:
                sign = np.ones(len(h))*-1
                t = []
                for i in range(len(h)):

                    t.append(alias_sample(
                        self.node_accept, self.node_alias))
                    
            sense_feats = self.sense_features[np.array(h)]
            adj = self.A[np.array(h), :]
            adj = adj[:, np.array(h)].todense()
            #assert adj.shape == (self.batch_size, self.batch_size)
            
            if self.order == 'all':
                yield ([np.array(h), np.array(t), adj, sense_feats],
                       [sign, sign, sense_feats, sense_feats])
            else:
                yield ([np.array(h), np.array(t), adj, sense_feats],
                       [sign, sense_feats, sense_feats])
            mod += 1
            mod %= mod_size
            if mod == 0:
                start_index = end_index
                end_index = min(start_index + self.batch_size, data_size)

            if start_index >= data_size:
                count += 1
                mod = 0
                h = []
                shuffle_indices = np.random.permutation(np.arange(data_size))
                start_index = 0
                end_index = min(start_index + self.batch_size, data_size)

    def get_embeddings(self,):
        self._embeddings = {}
        if self.order == 'first':
            embeddings = self.embedding_dict['first'].get_weights()[0]
        elif self.order == 'second':
            embeddings = self.embedding_dict['second'].get_weights()[0]
        else:
            embeddings = np.hstack((self.embedding_dict['first'].get_weights()[
                                   0], self.embedding_dict['second'].get_weights()[0]))
        idx2node = self.idx2node
        for i, embedding in enumerate(embeddings):
            self._embeddings[idx2node[i]] = embedding

        return self._embeddings

    def train(self, epochs = 1, initial_epoch = 0, verbose = 1, times = 1):
        batch_size = self.batch_size
        self.reset_training_config(batch_size, times)
        hist = self.model.fit(self.batch_it,
                                        epochs = epochs,
                                        initial_epoch = initial_epoch,
                                        steps_per_epoch = self.steps_per_epoch,
                                        verbose = verbose)

        return hist
    
    def _create_A_L(self, graph, node2idx):
        node_size = graph.number_of_nodes()
        A_data = []
        A_row_index = []
        A_col_index = []

        for edge in graph.edges():
            v1, v2 = edge
            edge_weight = graph[v1][v2].get('weight', 1)

            A_data.append(edge_weight)
            A_row_index.append(node2idx[v1])
            A_col_index.append(node2idx[v2])

        A = sp.csr_matrix((A_data, (A_row_index, A_col_index)), shape = (node_size, node_size))
        A_ = sp.csr_matrix((A_data + A_data, (A_row_index + A_col_index, A_col_index + A_row_index)),
                           shape=(node_size, node_size))

        D = sp.diags(A_.sum(axis=1).flatten().tolist()[0])
        L = D - A_
        return A, L


In [1]:
def create_alias_table(area_ratio):
    """
    :param area_ratio: sum(area_ratio)=1
    :return: accept,alias
    """
    l = len(area_ratio)
    accept, alias = [0] * l, [0] * l
    small, large = [], []
    area_ratio_ = np.array(area_ratio) * l
    for i, prob in enumerate(area_ratio_):
        if prob < 1.0:
            small.append(i)
        else:
            large.append(i)

    while small and large:
        small_idx, large_idx = small.pop(), large.pop()
        accept[small_idx] = area_ratio_[small_idx]
        alias[small_idx] = large_idx
        area_ratio_[large_idx] = area_ratio_[large_idx] - \
                                 (1 - area_ratio_[small_idx])
        if area_ratio_[large_idx] < 1.0:
            small.append(large_idx)
        else:
            large.append(large_idx)

    while large:
        large_idx = large.pop()
        accept[large_idx] = 1
    while small:
        small_idx = small.pop()
        accept[small_idx] = 1

    return accept, alias


def alias_sample(accept, alias):
    """
    :param accept:
    :param alias:
    :return: sample index
    """
    N = len(accept)
    i = int(np.random.random() * N)
    r = np.random.random()
    if r < accept[i]:
        return i
    else:
        return alias[i]